In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
cd /content/drive/MyDrive/Academic/Topics/AI/Machine\ Learning\ Dr.\ Montazeri/Project/ml_mda

/content/drive/MyDrive/Academic/Topics/AI/Machine Learning Dr. Montazeri/Project/ml_mda


In [26]:
!pip install torch_geometric

In [27]:
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.1.0+cu121.html

Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu121.html


# Requirements

In [28]:
import torch

from base import OptimizerConfig, cross_validation
from base import SimplePytorchData, SimplePytorchDataTrainTestSplit
from base import SimpleTrainer, SimpleTester
from src.config import Node2VecConfig, Node2VecOptimizerConfig, SimpleClassifierConfig
from src.config import GraphAutoEncoderConfig
from src.features import get_associations, get_node2vec_gae_pair_embedd_for_training_data
from src.models import SimpleMDAClassifier, SimpleMDAClassifierFactory
from src.utils import train_test_sampler, prj_logger

from torch_geometric.nn import GCNConv

In [29]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [30]:
import logging
import sys

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.StreamHandler(stream=sys.stdout)
    ],
    force=True
)

# Node2Vec + Graph Auto Encoder Embedding

## Config

In [31]:
# Node2Vec Config
node2vec_config = Node2VecConfig()
node2vec_config.embedding_dim = 128
node2vec_config.walk_length = 50
node2vec_config.context_size = 10
node2vec_config.walks_per_node = 10
node2vec_config.num_negative_samples = 1
node2vec_config.p = 1.0
node2vec_config.q = 1.0
node2vec_config.num_nodes = None
node2vec_config.sparse = True

# Node2Vec Optimizer Config
node2vec_optimizer_config = Node2VecOptimizerConfig()
node2vec_optimizer_config.exp_name = 'Node2VecOptimizer For All Nodes'
node2vec_optimizer_config.shuffle = True
node2vec_optimizer_config.num_workers = 2
node2vec_optimizer_config.lr = 0.01
node2vec_optimizer_config.device = device
node2vec_optimizer_config.report_size = 1000
node2vec_optimizer_config.optimizer = torch.optim.SparseAdam

In [32]:
# Graph Auto Encoder Config
gae_model_config = GraphAutoEncoderConfig()
gae_model_config.model_name = "Graph Auto Encoder Model"
gae_model_config.device = device
gae_model_config.hidden_dim = 32
gae_model_config.output_dim = 32
gae_model_config.num_layers = 3
gae_model_config.dropout = 0.5
gae_model_config.GCN = GCNConv

# Graph Auto Encoder Optimizer Config
gae_optimizer_config = OptimizerConfig()
gae_optimizer_config.optimizer = torch.optim.Adam
gae_optimizer_config.criterion = torch.nn.BCEWithLogitsLoss()
gae_optimizer_config.lr = 0.01
gae_optimizer_config.n_epoch = 80
gae_optimizer_config.exp_name = "Optimizer for Graph Auto Encoder"
gae_optimizer_config.device = device

## Embedding

In [33]:
md_embed = get_node2vec_gae_pair_embedd_for_training_data(node2vec_config, node2vec_optimizer_config, gae_model_config, gae_optimizer_config)

2024-02-09 08:23:35,040 [INFO] Calling get_node2vec_gae_pair_embedd on cuda device ...
2024-02-09 08:23:35,045 [INFO] Calling get_homogeneous_graph
2024-02-09 08:23:36,330 [INFO] homogeneous data : Data(x=[66911, 1], edge_index=[2, 633662])
2024-02-09 08:23:36,331 [INFO] Calling get_homogeneous_graph
2024-02-09 08:23:37,271 [INFO] homogeneous data : Data(x=[66911, 1], edge_index=[2, 633662])
2024-02-09 08:23:37,278 [INFO] Calling get_node2vec_embedd on cuda device ...
2024-02-09 08:23:37,280 [INFO] Creating Node2Vec model ...
2024-02-09 08:23:37,282 [INFO] Calling get_homogeneous_graph
2024-02-09 08:23:38,779 [INFO] homogeneous data : Data(x=[66911, 1], edge_index=[2, 633662])
2024-02-09 08:23:38,785 [INFO] Initialing MDNode2Vec with model_config {'model_name': None, 'embedding_dim': 128, 'walk_length': 50}
2024-02-09 08:23:39,015 [INFO] Training Node2Vec ...
2024-02-09 08:23:39,017 [INFO] Running Node2vecTrainer with Node2VecOptimizer For All Nodes
2024-02-09 08:23:39,019 [INFO] Creat

# Classification

## Data

In [34]:
associations = get_associations()
y = torch.tensor(associations['increased'].tolist(), dtype=torch.float32).reshape(-1, 1).to(device)

In [35]:
# Train Test Split
train_indices, test_indices = train_test_sampler(y.shape[0], 0.7)

data = SimplePytorchData(md_embed, y)
train_data = SimplePytorchData(md_embed[train_indices], y[train_indices])
test_data = SimplePytorchData(md_embed[test_indices], y[test_indices])

2024-02-09 08:30:17,607 [INFO] Initializing SimplePytorchData with X shape : torch.Size([898, 64]) and y shape : torch.Size([898, 1])
2024-02-09 08:30:17,613 [INFO] Initializing SimplePytorchData with X shape : torch.Size([628, 64]) and y shape : torch.Size([628, 1])
2024-02-09 08:30:17,615 [INFO] Initializing SimplePytorchData with X shape : torch.Size([270, 64]) and y shape : torch.Size([270, 1])


## Classifier

In [36]:
simple_classifier_config = SimpleClassifierConfig()
simple_classifier_config.model_name = "simple classifier"
simple_classifier_config.input_dim = md_embed.shape[1]
simple_classifier_config.hidden_dim = 32
simple_classifier_config.output_dim = 1
simple_classifier_config.num_layers = 3
simple_classifier_config.dropout = 0.1

In [37]:
mda_classifier = SimpleMDAClassifier(simple_classifier_config)

2024-02-09 08:30:17,634 [INFO] Initializing SimpleMDAClassifier with model : simple classifier
2024-02-09 08:30:17,635 [INFO] Initial SimpleMLP with 64 input dimension, 32 hidden dimension, 1 
            output dimension, 3 layers and with 0.1 dropout


## Train Test Approach

### Train

In [38]:
classifier_optimizer_config = OptimizerConfig()
classifier_optimizer_config.optimizer = torch.optim.Adam
classifier_optimizer_config.criterion = torch.nn.BCEWithLogitsLoss()
classifier_optimizer_config.lr = 0.01
classifier_optimizer_config.batch_size = 32
classifier_optimizer_config.n_epoch = 50
classifier_optimizer_config.exp_name = "adam optimizer"
classifier_optimizer_config.save = False
classifier_optimizer_config.save_path = None
classifier_optimizer_config.device = device
classifier_optimizer_config.report_size = 10  # batch to report ratio
classifier_optimizer_config.threshold = 0.5

In [39]:
train_result = SimpleTrainer().train(model=mda_classifier,
                                     data=train_data,
                                     config=classifier_optimizer_config)

2024-02-09 08:30:17,658 [INFO] Running Simple Trainer with config : adam optimizer
2024-02-09 08:30:17,659 [INFO] moving data and model to cuda
2024-02-09 08:30:17,697 [INFO] loss: 0.0212    [1,    10]
2024-02-09 08:30:17,728 [INFO] loss: 0.0210    [1,    20]
2024-02-09 08:30:17,756 [INFO] loss: 0.0202    [2,    10]
2024-02-09 08:30:17,786 [INFO] loss: 0.0214    [2,    20]
2024-02-09 08:30:17,811 [INFO] loss: 0.0203    [3,    10]
2024-02-09 08:30:17,837 [INFO] loss: 0.0191    [3,    20]
2024-02-09 08:30:17,865 [INFO] loss: 0.0192    [4,    10]
2024-02-09 08:30:17,893 [INFO] loss: 0.0196    [4,    20]
2024-02-09 08:30:17,919 [INFO] loss: 0.0185    [5,    10]
2024-02-09 08:30:17,945 [INFO] loss: 0.0192    [5,    20]
2024-02-09 08:30:17,981 [INFO] loss: 0.0182    [6,    10]
2024-02-09 08:30:18,018 [INFO] loss: 0.0192    [6,    20]
2024-02-09 08:30:18,046 [INFO] loss: 0.0186    [7,    10]
2024-02-09 08:30:18,073 [INFO] loss: 0.0184    [7,    20]
2024-02-09 08:30:18,100 [INFO] loss: 0.0176 

### Test

In [40]:
test_result = SimpleTester().test(model=mda_classifier,
                                  data=test_data,
                                  config=classifier_optimizer_config)

2024-02-09 08:30:20,520 [INFO] Running Simple Tester with config : adam optimizer
2024-02-09 08:30:20,526 [INFO] moving data and model to cuda
2024-02-09 08:30:20,547 [INFO] Result on Test Data : {'AUC': 0.8532368537339814, 'ACC': 0.7666666666666667, 'F1 Score': 0.7665097253297827, 'AUPR': 0, 'Loss': 0.503961142566469}


In [41]:
test_result.get_result()

{'AUC': 0.8532368537339814,
 'ACC': 0.7666666666666667,
 'F1 Score': 0.7665097253297827,
 'AUPR': 0,
 'Loss': 0.503961142566469}

## Cross Validation

In [42]:
trainer = SimpleTrainer()
tester = SimpleTester()
factory = SimpleMDAClassifierFactory(simple_classifier_config)
spliter = SimplePytorchDataTrainTestSplit(data)
cross_validation(k=5, data_size=data.X.shape[0], train_test_spliter=spliter, model_factory=factory,
                    trainer=trainer, tester=tester, config=classifier_optimizer_config)

2024-02-09 08:30:20,567 [INFO] Initializing SimpleMDAClassifierFactory with model : simple classifier
2024-02-09 08:30:20,570 [INFO] Initializing SimplePytorchDataTrainTestSplit
2024-02-09 08:30:20,571 [INFO] Start 5-fold Cross Validation with config : adam optimizer
2024-02-09 08:30:20,573 [INFO] ---- Fold 1 ----
2024-02-09 08:30:20,575 [INFO] Initializing SimplePytorchData with X shape : torch.Size([719, 64]) and y shape : torch.Size([719, 1])
2024-02-09 08:30:20,576 [INFO] Initializing SimplePytorchData with X shape : torch.Size([179, 64]) and y shape : torch.Size([179, 1])
2024-02-09 08:30:20,577 [INFO] Initializing SimpleMDAClassifier with model : simple classifier
2024-02-09 08:30:20,578 [INFO] Initial SimpleMLP with 64 input dimension, 32 hidden dimension, 1 
            output dimension, 3 layers and with 0.1 dropout
2024-02-09 08:30:20,580 [INFO] Running Simple Trainer with config : adam optimizer
2024-02-09 08:30:20,581 [INFO] moving data and model to cuda
2024-02-09 08:30:20